In [2]:
import xarray as xr
import pandas as pd
import logging

In [4]:
# Load the NetCDF datacube with chunking
ds = xr.open_dataset("/Users/vladimir/catalonia-wildfire-prediction/data/IberFire.nc", chunks="auto")

In [5]:
print(ds)

<xarray.Dataset> Size: 731GB
Dimensions:                                        (y: 920, x: 1188, time: 6241)
Coordinates:
  * x                                              (x) float64 10kB 2.675e+06...
  * y                                              (y) float64 7kB 2.492e+06 ...
  * time                                           (time) datetime64[ns] 50kB ...
Data variables: (12/261)
    x_index                                        (y, x) uint16 2MB dask.array<chunksize=(920, 1188), meta=np.ndarray>
    y_index                                        (y, x) uint16 2MB dask.array<chunksize=(920, 1188), meta=np.ndarray>
    is_spain                                       (y, x) uint16 2MB dask.array<chunksize=(920, 1188), meta=np.ndarray>
    is_fire                                        (time, y, x) uint8 7GB dask.array<chunksize=(1562, 230, 298), meta=np.ndarray>
    is_near_fire                                   (time, y, x) uint8 7GB dask.array<chunksize=(1562, 230, 298), meta=